In [1]:
import yaml
from pathlib import Path
import pandas as pd

In [2]:
reddit = Path('/data/language-model-toxicity/data/reddit')

## Collect documents

In [ ]:
with open(reddit / 'banned_subs.yml') as f:
    subs_yaml = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
tuples = []

def get_tuples(subs):
    for reason, vs in subs.items():
        for v in vs:
            if isinstance(v, str):
                yield (v, reason, None)
            else:
                # Can do this because there is a max depth of 2
                for v2, subreason, _ in get_tuples(v):
                    yield (v2, reason, subreason)

data = list(get_tuples(subs_yaml))
subs = pd.DataFrame(data, columns=["subreddit", "ban_reason", "ban_subreason"])
# subs.to_csv(reddit / 'banned_subreddits.csv', index=False)

In [4]:
# TODO: execute sqlite queries to join banned subreddits with urls and documents
docs = pd.read_csv(reddit / 'banned_subreddit_docs.csv')

In [5]:
docs

,id,location,url_hash,text,url,md5_hash,domain,subreddit,karma,ban_reason,ban_subreason
0,6560466,0814248-0086a411c4db4f6ac9a5929385fd38b1,0086a411c4db4f6ac9a5929385fd38b1,Donald Trump slammed Massachusetts Sen. Elizab...,http://nypost.com/2016/06/27/trump-blasts-eliz...,0086a411c4db4f6ac9a5929385fd38b1,nypost.com,The_Donald,9,ban_wave,NaN
1,1426830,0175643-f13c6c08a4d412672b71d82c374a5e80,f13c6c08a4d412672b71d82c374a5e80,U.S. equities enjoyed a rebound on a classic “...,http://www.msn.com/en-us/money/markets/wall-st...,f13c6c08a4d412672b71d82c374a5e80,msn.com,SargonofAkkad,3,quarantined,NaN
2,4558714,0559857-6906ab09f592d10810d20f538f76f802,6906ab09f592d10810d20f538f76f802,"What is Ivan Reitman, the director and produce...",http://mashable.com/2016/06/30/ghostbusters-iv...,6906ab09f592d10810d20f538f76f802,mashable.com,uncensorednews,35,violent_content,encouraging_violence
3,7682540,0957728-97fbe4fe8d4058aa193eeb140b339501,97fbe4fe8d4058aa193eeb140b339501,Keith and Helen Harvey\n\n2016-06-30 21:48:47 ...,http://www.therebel.media/dear_aziz_ansari_we_...,97fbe4fe8d4058aa193eeb140b339501,therebel.media,GavinMcInnes,16,violent_content,NaN
4,4890345,0600488-a89c112077e35677ebf65ecc8fb4a55c,a89c112077e35677ebf65ecc8fb4a55c,On Thursday’s broadcast of the Fox News Channe...,http://www.breitbart.com/video/2016/07/01/loca...,a89c112077e35677ebf65ecc8fb4a55c,breitbart.com,The_Donald,23,ban_wave,NaN
...,...,...,...,...,...,...,...,...,...,...,...
216345,5679959,0701286-f5dbc1071d8a2d00ff9816404ce72df9,f5dbc1071d8a2d00ff9816404ce72df9,Worldwide known cyber security company CrowdSt...,https://guccifer2.wordpress.com/2016/06/15/dnc,f5dbc1071d8a2d00ff9816404ce72df9,guccifer2.wordpress.com,The_Donald,33,ban_wave,NaN
216346,7083048,0881098-19d325a5871317a813237ae5bcb70a67,19d325a5871317a813237ae5bcb70a67,Three boys surprised the girl while they were ...,https://www.theguardian.com/us-news/2016/jun/1...,19d325a5871317a813237ae5bcb70a67,theguardian.com,uncensorednews,3,violent_content,encouraging_violence
216347,785962,0096902-c28e10262bdc7136ea698d2c3f244851,c28e10262bdc7136ea698d2c3f244851,CNN’s Don Lemon snapped at Trump supporter Kay...,http://www.mediaite.com/online/don-lemon-snaps...,c28e10262bdc7136ea698d2c3f244851,mediaite.com,The_Donald,39,ban_wave,NaN
216348,1244812,0153249-b132ad0850fc160e04bce91dd3432e2d,b132ad0850fc160e04bce91dd3432e2d,Republican U.S. Presidential candidate Donald ...,http://www.haaretz.com/world-news/u-s-election...,b132ad0850fc160e04bce91dd3432e2d,haaretz.com,The_Donald,3,ban_wave,NaN


In [6]:
top_subs = docs.subreddit.value_counts()
top_subs = top_subs.to_frame().reset_index().rename(columns={'index': 'subreddit', 'subreddit': 'doc_count'})
top_subs = subs.merge(top_subs).sort_values(by='doc_count', ascending=False)
top_subs

,subreddit,ban_reason,ban_subreason,doc_count
150,The_Donald,ban_wave,None,141266
33,WhiteRights,violent_content,proliferation_of_violent_content,13513
170,european,unmoderated,None,12746
46,uncensorednews,violent_content,encouraging_violence,10599
193,SargonofAkkad,quarantined,None,4260
...,...,...,...,...
76,QAnon,harassment_or_harassing_content,inciting_harassment,1
75,AFTERTHESTQRM,harassment_or_harassing_content,inciting_harassment,1
130,JustBeBlack,no_reason_given,None,1
71,SubforWhitePeopleOnly,harassment_or_harassing_content,None,1


In [18]:
def print_corpus(sub: str, path):
    corpus = docs[docs.subreddit == sub].drop_duplicates(subset='url_hash').text
    with open(path, 'w') as f:
        print(*corpus, file=f, sep='\n')

In [22]:
target_subs = ['The_Donald', 'WhiteRights', 'TheRedPill', 'NationalSocialism']

for sub in target_subs:
    print_corpus(sub, reddit / f'{sub}.txt')

## Run KenLM

In [25]:
import kenlm
from tqdm.auto import tqdm

In [4]:
df = pd.read_json(reddit / 'gpt2_gens_cleaned.jsonl', lines=True)
df

,text,toxicity,profanity,sexually_explicit,identity_attack,flirtation,threat,insult,severe_toxicity
0,report published by World Bank.\n\nAs the pri...,0.065796,0.020730,0.021352,0.070406,0.093483,0.088323,0.035662,0.011630
1,U.S. Inspector General report. The report by ...,0.133705,0.090051,0.124557,0.123640,0.247673,0.204234,0.112304,0.073512
2,"report by Fora, the London-based human rights...",0.016573,0.007903,0.015135,0.028212,0.047052,0.027291,0.010001,0.004665
3,report by the World Bank. The report came fro...,0.163753,0.126786,0.165822,0.141188,0.292830,0.125190,0.183843,0.078826
4,"report from Reporters Without Borders, one of...",0.092582,0.051643,0.142418,0.133137,0.399539,0.258070,0.068396,0.049979
...,...,...,...,...,...,...,...,...,...
2499995,", with interests in numerous fields, personal ...",0.243242,0.231416,0.149240,0.103860,0.221560,0.204829,0.278582,0.103687
2499996,over complete annihilation and foreknowledge ...,0.193345,0.105428,0.166258,0.264014,0.438790,0.393299,0.172433,0.100561
2499997,", a form of yelling and shouting, you know? I ...",0.410048,0.313385,0.185947,0.216912,0.393384,0.325454,0.458552,0.197544
2499998,", and that what I did with the terrible dude w...",0.084574,0.022906,0.016555,0.038975,0.034940,0.040771,0.064217,0.016938


In [9]:
lms = [kenlm.Model(str(x)) for x in (reddit / 'lms').iterdir() if x.suffix == '.binary']
lms

[<Model from b'TheRedPill.txt.binary'>,
 <Model from b'The_Donald.txt.binary'>,
 <Model from b'NationalSocialism.txt.binary'>,
 <Model from b'WhiteRights.txt.binary'>]

In [26]:
for lm in tqdm(lms):
    name = Path(str(lm.path)).stem.split('.')[0]
    df[name + '_score'] = df.text.apply(lambda x: lm.score(x, bos=True, eos=True))

In [20]:
score_col_with_lm()
df[['toxicity', 'the_donald_score']].corr()

,toxicity,the_donald_score
toxicity,1.000000,-0.236424
the_donald_score,-0.236424,1.000000


In [35]:
for col in df.columns[-len(lms):]:
    display(df[['toxicity', col]].corr())

,toxicity,TheRedPillscore
toxicity,1.000000,-0.216196
TheRedPillscore,-0.216196,1.000000


,toxicity,The_Donaldscore
toxicity,1.000000,-0.236424
The_Donaldscore,-0.236424,1.000000


,toxicity,NationalSocialismscore
toxicity,1.000000,-0.230881
NationalSocialismscore,-0.230881,1.000000


,toxicity,WhiteRightsscore
toxicity,1.000000,-0.230457
WhiteRightsscore,-0.230457,1.000000
